# Working with Open Model Zoo Models
This tutorial shows how to download a model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo), convert it to OpenVINO™ IR format, show information about the model, and benchmark the model.

## OpenVINO and Open Model Zoo Tools

OpenVINO and Open Model Zoo tools are listed in the table below.

| Tool             | Command             | Description                                             |
|:-----------------|:--------------------|:--------------------------------------------------------|
| Model Downloader | omz_downloader      | Download models from Open Model Zoo.                    |
| Model Converter  | omz_converter       | Convert Open Model Zoo models to OpenVINO's IR format.  |
| Info Dumper      | omz_info_dumper     | Print information about Open Model Zoo models.          |
| Benchmark Tool   | benchmark_app       | Benchmark model performance by computing inference time.|

## Preparation

### Model Name

Set `model_name` to the name of the Open Model Zoo model to use in this notebook. Refer to the list of [public](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md) and [Intel](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md) pre-trained models for a full list of models that can be used. Set `model_name` to the model you want to use.

In [ ]:
# model_name = "resnet-50-pytorch"
model_name = "mobilenet-v2-pytorch"

### Imports

In [ ]:
import json
import sys
from pathlib import Path

from IPython.display import Markdown, display
from openvino.runtime import Core

sys.path.append("../utils")
from notebook_utils import DeviceNotFoundAlert, NotebookAlert

### Settings and Configuration

Set the file and directory paths. By default, this notebook downloads models from Open Model Zoo to the `open_model_zoo_models` directory in your `$HOME` directory. On Windows, the $HOME directory is usually `c:\users\username`, on Linux `/home/username`. To change the folder, change `base_model_dir` in the cell below.

The following settings can be changed:

* `base_model_dir`: Models will be downloaded into the `intel` and `public` folders in this directory.
* `omz_cache_dir`: Cache folder for Open Model Zoo. Specifying a cache directory is not required for Model Downloader and Model Converter, but it speeds up subsequent downloads.
* `precision`: If specified, only models with this precision will be downloaded and converted.

In [ ]:
base_model_dir = Path("~/open_model_zoo_models").expanduser()
omz_cache_dir = Path("~/open_model_zoo_cache").expanduser()
precision = "FP16"

# Check if an iGPU is available on this system to use with Benchmark App.
ie = Core()
gpu_available = "GPU" in ie.available_devices

print(
    f"base_model_dir: {base_model_dir}, omz_cache_dir: {omz_cache_dir}, gpu_availble: {gpu_available}"
)

## Download a Model from Open Model Zoo



Specify, display and run the Model Downloader command to download the model.

In [ ]:
## Uncomment the next line to show help in omz_downloader which explains the command-line options.

# !omz_downloader --help

In [ ]:
download_command = (
    f"omz_downloader --name {model_name} --output_dir {base_model_dir} --cache_dir {omz_cache_dir}"
)
display(Markdown(f"Download command: `{download_command}`"))
display(Markdown(f"Downloading {model_name}..."))
! $download_command

## Convert a Model to OpenVINO IR format

Specify, display and run the Model Converter command to convert the model to OpenVINO IR format. Model conversion may take a while. The output of the Model Converter command will be displayed. When the conversion is successful, the last lines of the output will include: `[ SUCCESS ] Generated IR version 11 model.` For downloaded models that are already in OpenVINO IR format, conversion will be skipped.

In [ ]:
## Uncomment the next line to show Help in omz_converter which explains the command-line options.

# !omz_converter --help

In [ ]:
convert_command = f"omz_converter --name {model_name} --precisions {precision} --download_dir {base_model_dir} --output_dir {base_model_dir}"
display(Markdown(f"Convert command: `{convert_command}`"))
display(Markdown(f"Converting {model_name}..."))

! $convert_command

## Get Model Information

The Info Dumper prints the following information for Open Model Zoo models:

* Model name
* Description
* Framework that was used to train the model
* License URL
* Precisions supported by the model
* Subdirectory: the location of the downloaded model
* Task type

This information can be shown by running `omz_info_dumper --name model_name` in a terminal. The information can also be parsed and used in scripts. 

In the next cell, run Info Dumper and use `json` to load the information in a dictionary. 

In [ ]:
model_info_output = %sx omz_info_dumper --name $model_name
model_info = json.loads(model_info_output.get_nlstr())

if len(model_info) > 1:
    NotebookAlert(
        f"There are multiple IR files for the {model_name} model. The first model in the "
        "omz_info_dumper output will be used for benchmarking. Change "
        "`selected_model_info` in the cell below to select a different model from the list.",
        "warning",
    )

model_info

Having information of the model in a JSON file allows extracting the path to the model directory, and building the path to the OpenVINO IR file.

In [ ]:
selected_model_info = model_info[0]
model_path = (
    base_model_dir
    / Path(selected_model_info["subdirectory"])
    / Path(f"{precision}/{selected_model_info['name']}.xml")
)
print(model_path, "exists:", model_path.exists())

## Run Benchmark Tool

By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput values (frames per second). 

In [ ]:
## Uncomment the next line to show Help in benchmark_app which explains the command-line options.

# !benchmark_app --help

In [ ]:
benchmark_command = f"benchmark_app -m {model_path} -t 15"
display(Markdown(f"Benchmark command: `{benchmark_command}`"))
display(Markdown(f"Benchmarking {model_name} on CPU with async inference for 15 seconds..."))

! $benchmark_command

### Benchmark with Different Settings

The `benchmark_app` tool displays logging information that is not always necessary. A more compact result is achieved when the output is parsed with `json`.

The following cells show some examples of `benchmark_app` with different parameters. Below are some useful parameters:

- `-d` A device to use for inference. For example: CPU, GPU, MULTI. Default: CPU.
- `-t` Time expressed in number of seconds to run inference. Default: 60.
- `-api` Use asynchronous (async) or synchronous (sync) inference. Default: async.
- `-b` Batch size. Default: 1.


Run `! benchmark_app --help` to get an overview of all possible command-line parameters.

In the next cell, define the `benchmark_model()` function that calls `benchmark_app`. This makes it easy to try different combinations. In the cell below that, you display available devices on the system.

> **Note**: In this notebook, `benchmark_app` runs for 15 seconds to give a quick indication of performance. For more accurate performance, it is recommended to run inference for at least one minute by setting the `t` parameter to 60 or higher, and run `benchmark_app` in a terminal/command prompt after closing other applications. Copy the **benchmark command** and paste it in a command prompt where you have activated the `openvino_env` environment. 

In [ ]:
def benchmark_model(model_xml, device="CPU", seconds=60, api="async", batch=1):
    ie = Core()
    model_path = Path(model_xml)
    if ("GPU" in device) and ("GPU" not in ie.available_devices):
        DeviceNotFoundAlert("GPU")
    else:
        benchmark_command = f"benchmark_app -m {model_path} -d {device} -t {seconds} -api {api} -b {batch}"
        display(Markdown(f"**Benchmark {model_path.name} with {device} for {seconds} seconds with {api} inference**"))
        display(Markdown(f"Benchmark command: `{benchmark_command}`"))

        benchmark_output = %sx $benchmark_command
        print("command ended")
        benchmark_result = [line for line in benchmark_output
                            if not (line.startswith(r"[") or line.startswith("      ") or line == "")]
        print("\n".join(benchmark_result))

In [ ]:
ie = Core()

# Show devices available for OpenVINO Runtime
for device in ie.available_devices:
    device_name = ie.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

In [ ]:
benchmark_model(model_path, device="CPU", seconds=15, api="async")

In [ ]:
benchmark_model(model_path, device="AUTO", seconds=15, api="async")

In [ ]:
benchmark_model(model_path, device="GPU", seconds=15, api="async")

In [ ]:
benchmark_model(model_path, device="MULTI:CPU,GPU", seconds=15, api="async")